In [1]:
import os

%load_ext autoreload
%autoreload 1

import numpy as np
import lasagne as L

import sys
sys.path.append('../')

Using cuDNN version 5105 on context None
Preallocating 906/3020 Mb (0.300000) on cuda
Mapped name None to device cuda: GeForce GTX 780 (0000:01:00.0)


In [2]:
%aimport QANet

floatX == float32
device == cuda


In [3]:
glove_ver = '840B'

glove_path = '/pio/data/data/glove_vec/' + glove_ver + '/glove/'
glove_embs = np.load(glove_path + 'glove.' + glove_ver + '.300d.npy')
voc_size = glove_embs.shape[0]

In [27]:
glove_words_fname = 'glove.' + glove_ver + '.wordlist.pkl'
glove_words_path  = '/pio/data/data/glove_vec/' + glove_ver + '/glove/' + glove_words_fname
glove_words = np.load(glove_words_path)
NAW_token = glove_words.index('<not_a_word>')

# Weryfikacja modeli

In [5]:
squad_base_path = '/pio/data/data/squad/'
squad_path = squad_base_path + 'glove' + glove_ver + '/'

In [7]:
from squad_tools import load_squad_train, filter_empty_answers, trim_data

train_data = load_squad_train(squad_path)
train_data = filter_empty_answers(train_data)
train_data = trim_data(train_data, 300)

Only positive samples.


In [28]:
from squad_tools import load_squad_dev

dev_data = load_squad_dev(squad_base_path, squad_path, make_negative=True, NAW_token=NAW_token)

Adding NAW token to dev set.


In [29]:
%%time
qa_net = QANet.QANet(voc_size=voc_size,
                     dev_data=dev_data,
                     emb_init=glove_embs,
                     emb_dropout=True,
                     skip_train_fn=True,
                     train_unk=True,
                     negative=True,
                     prefetch_word_embs=True)

Building the model...
Using dropout after wiq calculation.
Compiling theano functions:
    get_intermediate_results_fn...
    get_start_probs_fn...
    get_end_probs_fn...
Done
CPU times: user 9.47 s, sys: 81.9 ms, total: 9.55 s
Wall time: 9.44 s


In [10]:
qa_net.load_params('trained_models/glove_vocab/wiki_negative/charemb_trainNAW_dropout_memory_crashed/charemb_trainNAW_dropout_ep5.npz', glove_embs)

In [8]:
qa_net.load_params('../models/840B/best/model.ep09.npz', glove_embs)

In [30]:
qa_net.load_params('../models/840B/best_neg/model.ep10.npz', glove_embs)

In [9]:
for batch in qa_net._iterate_minibatches(dev_data[-3:], 10, with_answer_inds=False):
    inter_res = qa_net.get_intermediate_results_fn(*batch)
    break

In [23]:
i = 9

zip(inter_res[2][i], dev_data[1][i][1])

[(0.20208944, u'what'),
 (0.57205999, u'does'),
 (0.0076373792, u'afc'),
 (0.056363955, u'stand'),
 (0.16111681, u'for'),
 (0.00073242665, u'?')]

In [5]:
qa_net.load_params('trained_models/glove6B/charemb_fixed_dropout2/charemb_fixed_dropout2_ep3.npz', glove_embs)

In [31]:
%%time
qa_net.calc_dev_f1(0) # best_neg neg

Calculating validation F1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
F1:  60.3167757659
EM:  53.6707663198
CPU times: user 1min, sys: 1.49 s, total: 1min 1s
Wall time: 1min 1s


60.316775765885865

In [12]:
%%time
qa_net.calc_dev_f1(0) # float32, intermediate

Calculating validation F1...
Done 2500 examples
Done 5000 examples
Done 7500 examples
Done 10000 examples
Predictions done
F1:  71.4700068683
EM:  61.9962157048
CPU times: user 1min, sys: 1.45 s, total: 1min 1s
Wall time: 1min 1s


71.47000686829753

In [ ]:
'''
best:
    Pos: F1: 71.47   Exact: 61.99
best_neg:
    Pos: F1: 71.07   Exact: 62.08
    Neg: F1: 60.31   Exact: 53.67
'''


'''
OLD

charemb_trainNAW_dropout_memory_crashed/charemb_trainNAW_dropout_ep5.npz
    Pos: F1: 68.12   Exact: 58.04
    Neg: F1: 60.17   Exact: 51.84
    
    Classification task:
        [[TN=9857  FP= 455]
         [FN=1275  TP=9295]]
    
charemb_glove_fixed_dropout_ep13.npz
    Pos: F1: 69.56   Exact: 59.33
'''